In [7]:
import json
from pathlib import Path

import pandas as pd

import sys

sys.path.append("../")
from frontend.ui_utils import *

In [8]:
with open("../backend/config.json", "r") as file:
    config = json.load(file)

In [9]:
data_metadata_path = Path(config["data_dir"]) / "all_dataset_description.csv"
data_metadata = pd.read_csv(data_metadata_path)
data_metadata.head()

,Unnamed: 0,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,...,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures,description,qualities,features,Combined_information
0,0,2,anneal,1,1,active,ARFF,684.0,7.0,8.0,...,39.0,898.0,898.0,22175.0,6.0,33.0,**Author**: Unknown. Donated by David Sterling...,"AutoCorrelation : 0.6064659977703456, CfsSubse...","0 : [0 - family (nominal)], 1 : [1 - product-t...","did - 2, name - anneal, version - 1, uploader ..."
1,1,3,kr-vs-kp,1,1,active,ARFF,1669.0,3.0,1527.0,...,37.0,3196.0,0.0,0.0,0.0,37.0,Author: Alen Shapiro\nSource: [UCI](https://ar...,"AutoCorrelation : 0.9990610328638497, CfsSubse...","0 : [0 - bkblk (nominal)], 1 : [1 - bknwy (nom...","did - 3, name - kr-vs-kp, version - 1, uploade..."
2,2,4,labor,1,1,active,ARFF,37.0,3.0,20.0,...,17.0,57.0,56.0,326.0,8.0,9.0,**Author**: Unknown\n**Source**: Collective Ba...,"AutoCorrelation : 0.75, CfsSubsetEval_Decision...","0 : [0 - duration (numeric)], 1 : [1 - wage-in...","did - 4, name - labor, version - 1, uploader -..."
3,3,5,arrhythmia,1,1,active,ARFF,245.0,13.0,2.0,...,280.0,452.0,384.0,408.0,206.0,74.0,"**Author**: H. Altay Guvenir, Burak Acar, Hald...","AutoCorrelation : 0.35476718403547675, CfsSubs...","0 : [0 - age (numeric)], 1 : [1 - sex (nominal...","did - 5, name - arrhythmia, version - 1, uploa..."
4,4,6,letter,1,1,active,ARFF,813.0,26.0,734.0,...,17.0,20000.0,0.0,0.0,16.0,1.0,**Author**: David J. Slate \n**Source**: [UCI...,"AutoCorrelation : 0.04090204510225511, CfsSubs...","0 : [0 - x-box (numeric)], 1 : [1 - y-box (num...","did - 6, name - letter, version - 1, uploader ..."


In [11]:
from structured_query.chroma_store_utilis import *

collec = load_chroma_metadata()

100%|██████████| 57/57 [00:00<00:00, 83.38it/s]


In [15]:
collec.get(ids=["10"])

{'ids': ['10'],
 'embeddings': None,
 'metadatas': [{'MajorityClassSize': 81.0,
   'MaxNominalAttDistinctValues': 8.0,
   'MinorityClassSize': 2.0,
   'NumberOfClasses': 4.0,
   'NumberOfFeatures': 19.0,
   'NumberOfInstances': 148.0,
   'NumberOfInstancesWithMissingValues': 0.0,
   'NumberOfMissingValues': 0.0,
   'NumberOfNumericFeatures': 3.0,
   'NumberOfSymbolicFeatures': 16.0,
   'Unnamed: 0': 8,
   'description': "**Author**:   \n**Source**: Unknown -   \n**Please cite**:   \n\nCitation Request:\n    This lymphography domain was obtained from the University Medical Centre,\n    Institute of Oncology, Ljubljana, Yugoslavia.  Thanks go to M. Zwitter and \n    M. Soklic for providing the data.  Please include this citation if you plan\n    to use this database.\n \n 1. Title: Lymphography Domain\n \n 2. Sources: \n     (a) See Above.\n     (b) Donors: Igor Kononenko, \n                 University E.Kardelj\n                 Faculty for electrical engineering\n                 Trzas

In [27]:
class ResponseParser:
    def __init__(self, query_type, apply_llm_before_rag=False):
        self.query_type = query_type
        self.paths = self.load_paths()
        self.rag_response = None
        self.llm_response = None
        self.apply_llm_before_rag = apply_llm_before_rag
        self.database_filtered = None
        self.structured_query_response = None

    def load_paths(self):
        """
        Description: Load paths from paths.json
        """
        with open("../frontend/paths.json", "r") as file:
            return json.load(file)




# Override the load_paths method
def custom_load_paths():
    with open(custom_paths_json_path, "r") as file:
        return json.load(file)


# Bind the custom load_paths method to the response_parser instance
response_parser.load_paths = custom_load_paths
response_parser.paths = response_parser.load_paths()

In [29]:
response_parser = ResponseParser(query_type="Dataset", apply_llm_before_rag=True)

response_parser

In [30]:
query_type = "Dataset"
query = "give me mushrom dataset with 25 classes"

In [32]:
structured_query = response_parser.fetch_structured_query(
    query_type=query_type, query=query
)

In [33]:
structured_query

[{'query': 'mushroom dataset with 25 classes',
  'filter': {'comparator': 'eq',
   'attribute': 'NumberOfClasses',
   'value': 25.0},
  'limit': None},
 {'filter': {'NumberOfClasses': {'$eq': 25.0}}}]

In [35]:
output = response_parser.fetch_rag_response(query_type, structured_query[0]["query"])

In [36]:
len(output["initial_response"])

30

In [37]:
filter_condition = structured_query[1]["filter"]
filter_condition = {"NumberOfClasses": {"$lte": 25.0}}

In [18]:
ids = []
for id in output["initial_response"]:
    ids.append(str(id))
ids

['627',
 '623',
 '626',
 '257',
 '245',
 '624',
 '636',
 '628',
 '253',
 '632',
 '633',
 '6419',
 '258',
 '761',
 '6414',
 '634',
 '255',
 '13',
 '249',
 '254',
 '328',
 '635',
 '813',
 '637',
 '629',
 '250',
 '246',
 '165',
 '506',
 '243']

In [38]:
database_filter = collec.get(ids=ids, where=filter_condition)
database_filter

['2578',
 '2322',
 '2063',
 '2213',
 '4199',
 '4245',
 '19052',
 '3527',
 '360',
 '3668',
 '2435',
 '2569',
 '19053',
 '4288',
 '3893',
 '2198',
 '4273',
 '2306',
 '7287',
 '2601',
 '3885',
 '3618',
 '3524',
 '2771',
 '2563',
 '1823',
 '2416',
 '2211',
 '2320',
 '3682']

In [39]:
collec.get(ids=ids, include=["metadatas"])

{'ids': ['3524', '3527', '3618', '3668', '3682', '3885', '3893'],
 'embeddings': None,
 'metadatas': [{'NumberOfClasses': 0.0,
   'NumberOfFeatures': 1026.0,
   'NumberOfInstances': 43.0,
   'NumberOfInstancesWithMissingValues': 0.0,
   'NumberOfMissingValues': 0.0,
   'NumberOfNumericFeatures': 1025.0,
   'NumberOfSymbolicFeatures': 1.0,
   'Unnamed: 0': 1755,
   'description': '**Author**: Dr Ivan Olier, Dr Jeremy Besnard, Dr Noureddin Sadawi, Dr Larisa Soldatova, Dr Crina Grosan, Prof Ross King, Dr Richard Bickerton, Prof Andrew Hopkins and Dr Willem van Hoorn  \n**Source**: MetaQSAR project - September 2015  \n**Please cite**:   \n\nThis dataset contains QSAR data (from ChEMBL version 17) showing activity values (unit is pseudo-pCI50) of several compounds on drug target TID: 102734, and it has 43 rows and 1026 features (including IDs and class feature: MOLECULE_CHEMBL_ID and MEDIAN_PXC50). The features represent FCFP 1024bit Molecular Fingerprints which were generated from SMILES s